# Import TensorFlow 2.x.

In [1]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

print(tf.__version__)

2.2.0


# Import TensorFlow datasets.
*   MNIST dataset

In [2]:
import tensorflow_datasets as tfds

# Import TensorFlow addons.
* Triplet semi-hard loss

In [3]:
import tensorflow_addons as tfa

# Load MNIST dataset.

### Load MNIST dataset.
* train split
* test split

In [4]:
train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


### Normalize dataset images.

In [5]:
def _normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.
    return (image, label)

### Create dataset batches.

In [6]:
buffer_size = 1024
batch_size = 32

In [7]:
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
train_dataset = train_dataset.map(_normalize_image)

test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(_normalize_image)

# Create the model.
* No activation (or default linear activation) on last layer
* L2 normalized embeddings.

In [8]:
model = models.Sequential([
    layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    layers.Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    layers.Dropout(0.3),

    layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    layers.Dropout(0.3),

    layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=2),

    layers.Dropout(0.3),
    
    layers.Flatten(),
    layers.Dense(256, activation=None),                      # No activation on final dense layer
    layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalized embeddings
])

### Show model summary.

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 128)       640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 128)       65664     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        16448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

# Train the model.

### Compile the model.

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tfa.losses.TripletSemiHardLoss())

### Train the model.

In [11]:
history = model.fit(train_dataset, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.4033
Epoch 2/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1773
Epoch 3/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1323
Epoch 4/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.1134
Epoch 5/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0995
Epoch 6/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0942
Epoch 7/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0850
Epoch 8/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0806
Epoch 9/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0731
Epoch 10/10
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0729


# Evaluate the model.
* Create the embeddings for the test dataset.



In [12]:
embeddings = model.predict(test_dataset)

# Save the embeddings for visualization in the embedding projector.

In [13]:
import io

np.savetxt("embeddings-vecs.tsv", embeddings, delimiter='\t')

meta_file = io.open('embeddings-meta.tsv', 'w', encoding='utf-8')
for image, labels in tfds.as_numpy(test_dataset):
    [meta_file.write(str(label) + "\n") for label in labels]
meta_file.close()

# Visualize using Embedding Projector.

Generated embedding vector and metadata files can be loaded and visualized using Embedding Projector available [here](https://projector.tensorflow.org).